In [367]:
import time
from selenium import webdriver
from random import randint, choice, uniform
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchFrameException, ElementClickInterceptedException

In [432]:
options = Options()
# chrome_options.add_argument("--headless")
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Living Room\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Windows\Temp\ipykernel_768\244793957.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=options)


In [283]:
def sign_up_for_yocket():
    driver.get('https://yocket.com/login')

In [408]:
def tap_and_select_random_option(base_locator,drop_locator_index,input_string=None):
    try:
        drop_down_locator = '#vs' + str(drop_locator_index) + '__listbox'
        drop_list_locator = '#vs' + str(drop_locator_index) + '__combobox > div.vs__selected-options > input'
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, base_locator))).click()
        if input_string :
            driver.find_element(By.CSS_SELECTOR,drop_list_locator).clear()
            driver.find_element(By.CSS_SELECTOR,drop_list_locator).send_keys(input_string)
            time.sleep(2)
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, drop_down_locator)))
        drop_down_options = get_all_options_from_dropdown(drop_down_locator)
#         print(drop_down_options)
        random_value = randint(0, len(drop_down_options[0])-1)
        if input_string :
            drop_down_options[0] = drop_down_options[0][:-1]
            if len(drop_down_options[0]) <=2 :
                random_value = randint(0, len(drop_down_options[0])-1)
            else : random_value = randint(0, len(drop_down_options[0])-2)
        vs_random_locator = '#vs' + str(drop_locator_index) + '__option-' + str(random_value)
#         print(vs_random_locator)
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, vs_random_locator))).click()
    except ElementClickInterceptedException:
        print(" cheking if chat_box is open for help ")
        if close_the_chat_box() : 
            print( " CLOSED SUCCESSFULLY")
            tap_and_select_random_option(base_locator,drop_locator_index,input_string=input_string)
        pass

In [409]:
def click_to_next_page(step=None):
    """Click's the next button 
        present at the bottom 
        of the page
    """
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > validationobserver > form > div.grid.grid-cols-2.gap-6.pt-12 > div.text-right > button')
#         driver.execute_script("arguments[0].click();", next_button)
        next_button.click()
    except :
        if( step == 1): 
            driver.refresh()
            time.sleep(2)
            tap_and_select_random_option("#country",'1')
        print("try completing the required mandatory fields on the page!, refreshing page")

In [430]:
def get_current_step(try_once = 1):
    """load's the curent step 
        number on a given page
    """
    try :
        try_once -=1
        step_info = driver.find_element(By.CSS_SELECTOR , '#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div.-mx-4.sm\:-mx-12.border-b-2 > div > div.flex.justify-between.space-x-6.sm\:mr-12.mt-4.lg\:mt-0 > div.text-gray-700.hidden.sm\:block')
        print(f"Currently at step -> {step_info.text.split(' ')[1]}")
        return int(step_info.text.split(' ')[1])
    except NoSuchElementException :
        print("not able to retrieve the step info , retrying after a second")
        time.sleep(1)
        if try_once >= 0 : get_current_step(try_once)

In [411]:
def get_all_options_from_dropdown(locator):
    """ Returns all the possible options 
        under the given dropdown 
    input : ccs_selector locator for the 
            required dropdown
    output : list of options
    """
    drop_down_options = []
    drop_down_list = driver.find_element(By.CSS_SELECTOR, locator)
    drop_down_options.append(drop_down_list.text.split('\n'))
#     print(drop_down_list.text)
    return drop_down_options

In [412]:
def go_back_a_step():
    step_back = driver.find_element(By.CSS_SELECTOR , '#__layout > div > main > div > div > div > validationobserver > form > div.grid.grid-cols-2.gap-6.pt-12 > div:nth-child(1) > button')
    step_back.click()

In [448]:
def close_the_chat_box():
    try:
        driver.switch_to.frame("siqiframe")
        # driver.switch_to.default_content()
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'body > div.win_close.sqico-larrow'))).click()
        time.sleep(2)
        driver.switch_to.default_content()
        print("FOUND THE CHAT BOX INTERCEPTING THE MAIN PAGE")
        return True
    except (NoSuchFrameException,NoSuchElementException,ElementClickInterceptedException):
        driver.switch_to.default_content()
        print("No such frame , or chat box is colsed already!")
        pass

In [414]:
def update_the_score(locator, score):
    try:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, locator))).send_keys(Keys.CONTROL + "a")
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, locator))).send_keys(Keys.DELETE)
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, locator))).send_keys(score)
    except :
        print("Locator issue!, check the locator asssigned")

In [358]:
root_page = 'https://yocket.com'
base_page  = 'https://yocket.com/college-finder/masters'
driver.get(base_page)

In [359]:
get_current_step()

Currently at step -> 1


1

In [362]:
tap_and_select_random_option("#country",'1')

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x01087413+2389011]
	Ordinal0 [0x01019F61+1941345]
	Ordinal0 [0x00F0C658+837208]
	Ordinal0 [0x00F391DD+1020381]
	Ordinal0 [0x00F3949B+1021083]
	Ordinal0 [0x00F66032+1204274]
	Ordinal0 [0x00F54194+1130900]
	Ordinal0 [0x00F64302+1196802]
	Ordinal0 [0x00F53F66+1130342]
	Ordinal0 [0x00F2E546+976198]
	Ordinal0 [0x00F2F456+980054]
	GetHandleVerifier [0x01239632+1727522]
	GetHandleVerifier [0x012EBA4D+2457661]
	GetHandleVerifier [0x0111EB81+569713]
	GetHandleVerifier [0x0111DD76+566118]
	Ordinal0 [0x01020B2B+1968939]
	Ordinal0 [0x01025988+1989000]
	Ordinal0 [0x01025A75+1989237]
	Ordinal0 [0x0102ECB1+2026673]
	BaseThreadInitThunk [0x76D26A14+36]
	RtlInitializeExceptionChain [0x7758AB2F+143]
	RtlInitializeExceptionChain [0x7758AAFA+90]


In [385]:
def check_if_proper_page(desired_step):
    if get_current_step() == desired_step :
        print("Got into proper page for desired step")
        return True
    elif get_current_step() > desired_step: 
        go_back_a_step()
        print("Stepping a page back")
        check_if_proper_page(desired_step)
    else :
        print("Please complete the mandatory information over current page before navigating to next")
        return False

In [433]:
def first_page(max_tries=3):
    try:
        max_tries -= 1
        check_if_proper_page(1)
        tap_and_select_random_option("#country",'1')
        print("Selected random country")
        close_the_chat_box()
        tap_and_select_random_option("#area_of_study",'2','eng')
        print("Selected random aread_of_study")
        close_the_chat_box()
        click_to_next_page()
        print("navigated to next page")
        return True
    except TimeoutException :
        driver.get(base_page)
        if (max_tries >= 0):
            first_page(max_tries)
        else :
            print("Maximum tries reached for first page ")

In [383]:
first_page()

Currently at step -> 2
Currently at step -> 2
Stepping a page back
Currently at step -> 1
Got into proper page and step
Currently at step -> 1
Got into proper page and step
United States
Canada
Australia & New Zealand
United Kingdom
European Union
#vs1__option-1
Selected random country
Electrical and Computer Engineering
Mechanical Engineering
Civil Engineering
Industrial Engineering
Engineering Management Mem
Electrical and Electronics Engineering
Aerospace Engineering
Civil and Environmental Engineering
Biomedical Engineering
Software Engineering
Mechatronics Robotics and Automation Engineering
Manufacturing Engineering
Computer Engineering
Automotive Engineering
Biochemical Engineering
Electrical Engineering
Engineering Science
Construction Engineering
Materials Engineering
Systems Engineering
Petroleum Engineering
Mining and Mineral Engineering
Engineering General
Engineering Design
Biological Biosystems Engineering
Environmental Engineering Technology
English
Process Engineering
E

In [436]:
def second_page(max_tries=2):
    try:
        max_tries -= 1
        if not check_if_proper_page(2):
            print("Filing out the first step first !")
            first_page()
        tap_and_select_random_option("#college",'4','tech')
        print("Selected random college")
        tap_and_select_random_option("#major",'6','Elec')
        print("Selected random major_of_study")
        driver.find_element(By.CSS_SELECTOR,'#marks').clear()
        random_cgpa = round(uniform(5,10), 2)
        update_the_score('#marks',random_cgpa)
        close_the_chat_box()
        click_to_next_page()
        print("clicked on next button ")
        return True
    except TimeoutException :
        driver.refresh()
        time.sleep(2)
        if (max_tries >= 0):
            second_page(max_tries)
        else :
            print("Maximum tries reached for second page ")

In [390]:
second_page()    

Currently at step -> 2
Got into proper page for desired step
Apex International Institute of Technology
Bal Krishna Institute of Technology
Gurukul Institute of Engineering & Technology
Maharishi Arvind College of Engineering & Technology
Maharishi Arvind International Institute of Technology
Modi Institute of Technology
Sardar Patel College of Technology
Shri Balaji Institute of Technology & Mgmt.
University Institute of Technology
Lakshmi Narain College of Technology
tech
#vs4__option-7
Selected random college
Computer Networking and Telecommunications
Computer Science Electronics
Electrical and Computer Engineering
Electrical and Electronics Engineering
Electrical and Power Engineering
Electrical Electronics and Communications Engineering
Electrical Engineering
Electrician
Electronics and Communication Engineering
Electronics Engineering
Telecommunication Engineering
Telecommunications Engineering and Technology
#vs6__option-3
Selected random major_of_study
clicked to next button 


True

In [391]:
exam_dict = {
    'ielts' : '2',
    'pte' : '3',
    'toefl' : '1'
    }

In [392]:
aptitude_dict = {
    'gre' : '1',
    'gmat' : '2',
    'none' : '3'
}

In [393]:
def fill_the_english_test_scores():
    english_test_name, english_locator_index = choice(list(exam_dict.items()))
    english_locator = ' #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pr-4 > div.grid.grid-cols-3.sm\:grid-cols-3.gap-2.justify-center.items-center.pt-2 > div:nth-child(' + str(english_locator_index) + ') > div' 
    print(english_test_name, english_locator)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, english_locator))).click()
    english_score = 0
    input_value_locator = '#'+ english_test_name + '_overall_score'
    print(input_value_locator)
    if( english_test_name == 'toefl') :
        english_score = randint(60,120)
    elif english_test_name == 'pte' :
        english_score = randint(45,90)
    elif english_test_name == 'ielts' :
        english_score = randint(5,9)
    else  :
        print( "select the proper exam section ! ")
    update_the_score(input_value_locator, english_score)

In [394]:
def fill_the_aptitude_test_scores():
    aptitude_test_name, aptitude_locator_index = choice(list(aptitude_dict.items()))
    aptitude_locator = '#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pl-8 > div.pt-2 > div.grid.grid-cols-3.md\:grid-cols-3.gap-2.justify-center.items-center > div:nth-child(' + str(aptitude_locator_index) + ') > div'
    print(aptitude_test_name, aptitude_locator)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, aptitude_locator))).click()
    if( aptitude_test_name == 'gre') :
#         aptitude_score = [randint(85,170),randint(85,170),randint(3,6)]
        update_the_score('#verbal_score', randint(130,170))
        update_the_score('#quant_score', randint(130,170))
        update_the_score('#awa_score', randint(3,6))
    elif aptitude_test_name == 'gmat' :
        update_the_score('#total_gmat_score', randint(450,800))
    elif aptitude_test_name == 'none' :
        aptitude_score = None
    else  :
        print( "select the proper exam section ! ")

In [437]:
def third_page(max_tries=1):
    try:
        max_tries -= 1
        if not check_if_proper_page(3):
            print("Filing out the second step first !")
            second_page()
        fill_the_english_test_scores()
        print("Filled random scores for random english test")
        fill_the_aptitude_test_scores()
        print("Filled random aptitude scores")
        close_the_chat_box()
        click_to_next_page()
        print("clicked on next button ")
        return True
    except TimeoutException :
        driver.refresh()
        time.sleep(2)
        if (max_tries >= 0):
            third_page(max_tries)
        else :
            print("Maximum tries reached for third page ")

In [396]:
third_page()

Currently at step -> 3
Got into proper page for desired step
ielts  #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pr-4 > div.grid.grid-cols-3.sm\:grid-cols-3.gap-2.justify-center.items-center.pt-2 > div:nth-child(2) > div
#ielts_overall_score
Filled random scores for random english test
gmat #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pl-8 > div.pt-2 > div.grid.grid-cols-3.md\:grid-cols-3.gap-2.justify-center.items-center > div:nth-child(2) > div
Filled random aptitude scores
clicked to next button 


True

In [438]:
def fourth_page(max_tries=1):
    try:
        max_tries -= 1
        if not check_if_proper_page(4):
            print("Filing out the third step first !")
            third_page()
        update_the_score('#work_exp',randint(5,50))
        print("entered random scores for work experience")
        update_the_score('#project',randint(0,10))
        print("entered random project's done")
        close_the_chat_box()
        click_to_next_page()
        print("clicked on next button ")
        return True
    except TimeoutException :
        driver.refresh()
        time.sleep(2)
        if (max_tries >= 0):
            fourth_page(max_tries)
        else :
            print("Maximum tries reached for fourth page ")

In [400]:
fourth_page()

Currently at step -> 4
Got into proper page for desired step
entered random scores for work experience
entered random project's done
clicked on next button 


True

In [401]:
relevance_dict = {
    'relevance' : '0',
    'cost' : '1'
}

In [464]:
def get_list_of_all_universities(locator):
    output = []
    universities = driver.find_element(By.CSS_SELECTOR, locator)
#     print(universities.text)
    raw_university = universities.text.split('Shortlist')[1:]
    for r in raw_university:
        output.append(r.split('\n')[1:])
    return output

In [485]:
def check_if_results_zero(locator):
    try : 
        result = driver.find_element(By.CSS_SELECTOR, locator)
        if result.text == 'Uh-oh! That spell didn’t work': return True
        else: return False
    except :
        print ('seomthing wrong with the locator')

Uh-oh! That spell didn’t work


In [486]:
def scrape_on_50_opacity():
    if driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > p').text == "Ambitious Universities (0)" :
#         error_message = driver.find_element(By.CSS_SELECTOR,'#__layout > div > main > div > div > div > div.text-center.lg\:text-left.text-gray-700 > div > p.text-xl.text-gray-900.text-opacity-75.font-semibold').text
        print("Our search couldn't help find a better match ! , here is the message ")
#         print(error_message)
        print("Sorry, we couldn’t find universities under the ambitious category that match your profile."
              "Try our Premium plans (spells) and get expert guidance! . checking for any Target Universities"
              "\nHere's your other options! TARGET Universities")
    else :
        val  = Select(driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > div:nth-child(5) > div.flex.justify-between.items-center > div > select'))
        val.select_by_value(relevance_dict['cost'])
        ambitious_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(5) > div.grid.grid-cols-1.sm\:grid-cols-2.lg\:grid-cols-3.gap-4.sm\:gap-6')
        print(ambitious_universities)
        return ambitious_universities
    target_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(7) > div')
#     safe_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(9) > div > div')
    safe_universities = get_list_of_all_universities('#__layout > div > main > div > div > div > div:nth-child(9) > div')
    
    print(target_universities)
    print("\nHere's your other options! Safe Universities")
    print(safe_universities)
    return target_universities

In [462]:
#__layout > div > main > div > div > div > div > p.text-2xl.md\:text-3xl.text-gray-900.text-opacity-75.font-semibold

In [488]:
def final_result(): 
    if check_if_results_zero('#__layout > div > main > div > div > div > div > p.text-2xl.md\:text-3xl.text-gray-900.text-opacity-75.font-semibold'):
        print(" FOUND ZERO RESULTS , PROBABLY YOU NEED TO RECONSIDER YUOR CARRER OPTION")
        pass
    else : 
        Ideal_match = scrape_on_50_opacity()
        print(" IDEAL UNIVERSITY FOUND IS {}".format (Ideal_match[0][1]))

In [405]:
def get_to_final_page():
    driver.get(base_page)
    first_page()
    second_page()
    third_page()
    fourth_page()

In [488]:
get_to_final_page()
final_result()

Currently at step -> 1
Got into proper page for desired step
 cheking if chat_box is open for help 
No such frame , or chat box is colsed already!
Selected random country
No such frame , or chat box is colsed already!
 cheking if chat_box is open for help 
No such frame , or chat box is colsed already!
Selected random aread_of_study
No such frame , or chat box is colsed already!
try completing the required mandatory fields on the page!, refreshing page
navigated to next page
Currently at step -> 1
Currently at step -> 1
Please complete the mandatory information over current page before navigating to next
Filing out the first step first !
Currently at step -> 1
Got into proper page for desired step
 cheking if chat_box is open for help 
No such frame , or chat box is colsed already!
Selected random country
No such frame , or chat box is colsed already!
 cheking if chat_box is open for help 
No such frame , or chat box is colsed already!
Selected random aread_of_study
No such frame , or 

KeyboardInterrupt: 

Exception ignored in: Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "C:\Users\Living Room\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\iostream.py", line 531, in write
    self.pub_thread.schedule(lambda: self._buffer.write(string))
  File "C:\Users\Living Room\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\iostream.py", line 216, in schedule
    self._event_pipe.send(b'')
  File "C:\Users\Living Room\AppData\Local\Programs\Python\Python310\lib\site-packages\zmq\sugar\socket.py", line 547, in send
    return super(Socket, self).send(data, flags=flags, copy=copy, track=track)
  File "zmq\backend\cython\socket.pyx", line 718, in zmq.backend.cython.socket.Socket.send
  File "zmq\backend\cython\socket.pyx", line 765, in zmq.backend.cython.socket.Socket.send
  File "zmq\backend\cython\socket.pyx", line 242, in zmq.backend.cython.socket._send_copy
  File "zmq\backend\cython\che

KeyboardInterrupt: 

In [345]:
tap_and_select_random_option("#area_of_study",'2','eng')

Electrical and Computer Engineering
Mechanical Engineering
Civil Engineering
Industrial Engineering
Engineering Management Mem
Electrical and Electronics Engineering
Aerospace Engineering
Civil and Environmental Engineering
Biomedical Engineering
Software Engineering
Mechatronics Robotics and Automation Engineering
Manufacturing Engineering
Computer Engineering
Automotive Engineering
Biochemical Engineering
Electrical Engineering
Engineering Science
Construction Engineering
Materials Engineering
Systems Engineering
Petroleum Engineering
Mining and Mineral Engineering
Engineering General
Engineering Design
Biological Biosystems Engineering
Environmental Engineering Technology
English
Process Engineering
Engineering Physics
Chemical and Biological Engineering
Energy Systems Engineering
Nuclear Engineering Technology
Engineering and Technology
Financial Engineering
Water Resources Engineering
Geological Engineering
#vs2__option-22


In [346]:
click_to_next_page()

In [349]:
tap_and_select_random_option("#college",'4','tech')

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x01087413+2389011]
	Ordinal0 [0x01019F61+1941345]
	Ordinal0 [0x00F0C658+837208]
	Ordinal0 [0x00F391DD+1020381]
	Ordinal0 [0x00F3949B+1021083]
	Ordinal0 [0x00F66032+1204274]
	Ordinal0 [0x00F54194+1130900]
	Ordinal0 [0x00F64302+1196802]
	Ordinal0 [0x00F53F66+1130342]
	Ordinal0 [0x00F2E546+976198]
	Ordinal0 [0x00F2F456+980054]
	GetHandleVerifier [0x01239632+1727522]
	GetHandleVerifier [0x012EBA4D+2457661]
	GetHandleVerifier [0x0111EB81+569713]
	GetHandleVerifier [0x0111DD76+566118]
	Ordinal0 [0x01020B2B+1968939]
	Ordinal0 [0x01025988+1989000]
	Ordinal0 [0x01025A75+1989237]
	Ordinal0 [0x0102ECB1+2026673]
	BaseThreadInitThunk [0x76D26A14+36]
	RtlInitializeExceptionChain [0x7758AB2F+143]
	RtlInitializeExceptionChain [0x7758AAFA+90]


In [348]:
tap_and_select_random_option("#major",'6','Elec')

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x01087413+2389011]
	Ordinal0 [0x01019F61+1941345]
	Ordinal0 [0x00F0C658+837208]
	Ordinal0 [0x00F391DD+1020381]
	Ordinal0 [0x00F3949B+1021083]
	Ordinal0 [0x00F66032+1204274]
	Ordinal0 [0x00F54194+1130900]
	Ordinal0 [0x00F64302+1196802]
	Ordinal0 [0x00F53F66+1130342]
	Ordinal0 [0x00F2E546+976198]
	Ordinal0 [0x00F2F456+980054]
	GetHandleVerifier [0x01239632+1727522]
	GetHandleVerifier [0x012EBA4D+2457661]
	GetHandleVerifier [0x0111EB81+569713]
	GetHandleVerifier [0x0111DD76+566118]
	Ordinal0 [0x01020B2B+1968939]
	Ordinal0 [0x01025988+1989000]
	Ordinal0 [0x01025A75+1989237]
	Ordinal0 [0x0102ECB1+2026673]
	BaseThreadInitThunk [0x76D26A14+36]
	RtlInitializeExceptionChain [0x7758AB2F+143]
	RtlInitializeExceptionChain [0x7758AAFA+90]


In [ ]:
driver.find_element(By.CSS_SELECTOR,'#marks').clear()
random_cgpa = round(uniform(5,10), 2)
update_the_score('#marks',random_cgpa)

In [ ]:
click_to_next_page()

In [ ]:
fill_the_english_test_scores()

In [ ]:
fill_the_aptitude_test_scores()

In [ ]:
click_to_next_page()

In [ ]:
update_the_score('#work_exp',randint(5,50))

In [ ]:
update_the_score('#project',randint(0,10))

In [443]:
click_to_next_page()

try completing the required mandatory fields on the page!, refreshing page


In [444]:
driver.refresh()

In [341]:
driver.execute_script("document.body.style.zoom = '40%'")
time.sleep(5)

In [321]:
scrape_on_50_opacity()

Shortlist
University of Technology Sydney
Australia
Masters Good Manufacturing Practice
A$71,520/year
24 months
See all admits
Get Guidance
Shortlist
University of Melbourne
Australia
MS Industrial Engineering
A$91,709/year
24 months
See all admits
Get Guidance


[['University of Technology Sydney', 'Australia', 'Masters Good Manufacturing Practice', 'A$71,520/year', '24 months', 'See all admits', 'Get Guidance', ''], ['University of Melbourne', 'Australia', 'MS Industrial Engineering', 'A$91,709/year', '24 months', 'See all admits', 'Get Guidance']]
[]
[]


In [280]:
#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div.grid.grid-cols-3.gap-6.mt-8 > div:nth-child(2) > div
val  = Select(driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > div:nth-child(5) > div.flex.justify-between.items-center > div > select'))
val.select_by_value(relevance_dict['cost'])  

In [64]:
click_to_next_page()

In [467]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#country"))).click()
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#vs1__listbox")))
country_options = get_all_options_from_dropdown('#vs1__listbox')

United States
Canada
Australia & New Zealand
United Kingdom
European Union


In [182]:
print(country_options)

[['United States', 'Canada', 'Australia & New Zealand', 'United Kingdom', 'European Union']]


In [470]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#area_of_study"))).click()
driver.find_element(By.CSS_SELECTOR,'#vs2__combobox > div.vs__selected-options > input').clear()
driver.find_element(By.CSS_SELECTOR,'#vs2__combobox > div.vs__selected-options > input').send_keys('data')
# time.sleep(2)
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#vs2__listbox")))
stream_options = get_all_options_from_dropdown('#vs2__listbox')

Data Science and Business Intelligence
Data Processing Technology


In [122]:
print(stream_options)

[['Data Science and Business Intelligence', 'Data Processing Technology', 'Big Data Analytics Applied Modelling and Quantitative Methods Non Thesis', 'Big Data Analytic and Information Technology']]


In [468]:

# print(len(country_options[0]))
random_value = randint(0, len(country_options[0])-1)
vs_1_locator = '#vs1__option-' + str(random_value)
print(vs_1_locator)

#vs1__option-3


In [471]:

# print(len(country_options[0]))
random_value = randint(0, len(stream_options[0])-1)
vs_2_locator = '#vs2__option-' + str(random_value)
print(vs_2_locator)

#vs2__option-0


In [469]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, vs_1_locator))).click()

In [472]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, vs_2_locator))).click()

In [166]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'body > div'))).click()

In [168]:
driver.switchTo().alert().dismiss();

AttributeError: 'WebDriver' object has no attribute 'switchTo'

In [473]:
click_to_next_page()


try completing the required mandatory fields on the page!


## second page

In [215]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#college"))).click()
driver.find_element(By.CSS_SELECTOR,'#vs4__combobox > div.vs__selected-options > input').clear()
driver.find_element(By.CSS_SELECTOR,'#vs4__combobox > div.vs__selected-options > input').send_keys('Tech')
# driver.find_element(By.CSS_SELECTOR,'#vs4__combobox > div.vs__selected-options > input').clear()
time.sleep(2)
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#vs4__listbox")))
college_options = get_all_options_from_dropdown('#vs4__listbox')

Apex International Institute of Technology
Bal Krishna Institute of Technology
Gurukul Institute of Engineering & Technology
Maharishi Arvind College of Engineering & Technology
Maharishi Arvind International Institute of Technology
Modi Institute of Technology
Sardar Patel College of Technology
Shri Balaji Institute of Technology & Mgmt.
University Institute of Technology
Lakshmi Narain College of Technology
Tech


In [218]:
print(college_options[0][:-1])

['Apex International Institute of Technology', 'Bal Krishna Institute of Technology', 'Gurukul Institute of Engineering & Technology', 'Maharishi Arvind College of Engineering & Technology', 'Maharishi Arvind International Institute of Technology', 'Modi Institute of Technology', 'Sardar Patel College of Technology', 'Shri Balaji Institute of Technology & Mgmt.', 'University Institute of Technology', 'Lakshmi Narain College of Technology']


In [219]:

# print(len(country_options[0]))
random_value = randint(0, len(college_options[0])-2) ## index -1 due to the fact the entered key is present at last
vs_4_locator = '#vs4__option-' + str(random_value)
print(vs_4_locator)

#vs4__option-5


In [220]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, vs_4_locator))).click()

In [ ]:
#major

In [221]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#major"))).click()
driver.find_element(By.CSS_SELECTOR,'#vs6__combobox > div.vs__selected-options > input').clear()
driver.find_element(By.CSS_SELECTOR,'#vs6__combobox > div.vs__selected-options > input').send_keys('Elec')
# driver.find_element(By.CSS_SELECTOR,'#vs4__combobox > div.vs__selected-options > input').clear()
time.sleep(2)
WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#vs6__listbox")))
major_options = get_all_options_from_dropdown('#vs6__listbox')

Computer Networking and Telecommunications
Computer Science Electronics
Electrical and Computer Engineering
Electrical and Electronics Engineering
Electrical and Power Engineering
Electrical Electronics and Communications Engineering
Electrical Engineering
Electrician
Electronics and Communication Engineering
Electronics Engineering
Telecommunication Engineering
Telecommunications Engineering and Technology


In [222]:

# print(len(country_options[0]))
random_value = randint(0, len(major_options[0])-1) 
vs_6_locator = '#vs6__option-' + str(random_value)
print(vs_6_locator)

#vs6__option-5


In [223]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, vs_6_locator))).click()

In [264]:
driver.find_element(By.CSS_SELECTOR,'#marks').clear()
# WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#marks"))).click()
# driver.find_element(By.CSS_SELECTOR,'#marks').clear()
random_cgpa = round(uniform(5,10), 2)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#marks"))).send_keys(Keys.CONTROL + "a")
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#marks"))).send_keys(Keys.DELETE)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#marks"))).send_keys(random_cgpa)

In [265]:
click_to_next_page()

## Third Page

In [305]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, exam_dict['pte']))).click()

teofl #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pr-4 > div.grid.grid-cols-3.sm\:grid-cols-3.gap-2.justify-center.items-center.pt-2 > div:nth-child(1) > div


In [335]:
fill_the_english_test_scores()

toefl #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pr-4 > div.grid.grid-cols-3.sm\:grid-cols-3.gap-2.justify-center.items-center.pt-2 > div:nth-child(1) > div
#toefl_overall_score


In [343]:
l= [randint(85,170),randint(85,170),randint(3,6)]
print(l[0])

148


In [366]:
fill_the_aptitude_test_scores()

gmat #__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div:nth-child(2) > div > div.col-span-2.sm\:col-span-1.sm\:pl-8 > div.pt-2 > div.grid.grid-cols-3.md\:grid-cols-3.gap-2.justify-center.items-center > div:nth-child(2) > div


In [367]:
click_to_next_page()

## fourth page

In [371]:
ele = driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div.grid.grid-cols-3.gap-6.mt-8 > div:nth-child(2) > div')

## Final page

In [279]:
#__layout > div > main > div > div > div > div:nth-child(5) > div.flex.justify-between.items-center > div > select
val  = Select(driver.find_element(By.CSS_SELECTOR, '#__layout > div > div.w-full > div > main > div > div > div > div:nth-child(4) > div.flex.justify-between.items-center > div > select'))
val.select_by_value(relevance_dict['cost'])            

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__layout > div > div.w-full > div > main > div > div > div > div:nth-child(4) > div.flex.justify-between.items-center > div > select"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x01087413+2389011]
	Ordinal0 [0x01019F61+1941345]
	Ordinal0 [0x00F0C658+837208]
	Ordinal0 [0x00F391DD+1020381]
	Ordinal0 [0x00F3949B+1021083]
	Ordinal0 [0x00F66032+1204274]
	Ordinal0 [0x00F54194+1130900]
	Ordinal0 [0x00F64302+1196802]
	Ordinal0 [0x00F53F66+1130342]
	Ordinal0 [0x00F2E546+976198]
	Ordinal0 [0x00F2F456+980054]
	GetHandleVerifier [0x01239632+1727522]
	GetHandleVerifier [0x012EBA4D+2457661]
	GetHandleVerifier [0x0111EB81+569713]
	GetHandleVerifier [0x0111DD76+566118]
	Ordinal0 [0x01020B2B+1968939]
	Ordinal0 [0x01025988+1989000]
	Ordinal0 [0x01025A75+1989237]
	Ordinal0 [0x0102ECB1+2026673]
	BaseThreadInitThunk [0x76D26A14+36]
	RtlInitializeExceptionChain [0x

In [419]:
ambitious_universities = get_list_of_all_universities('#__layout > div > div.w-full > div > main > div > div > div > div:nth-child(4) > div.grid.grid-cols-1.sm\:grid-cols-2.lg\:grid-cols-3.gap-4.sm\:gap-6')
target_universities = get_list_of_all_universities('#__layout > div > div.w-full > div > main > div > div > div > div:nth-child(6) > div')
safe_universities = get_list_of_all_universities('#__layout > div > div.w-full > div > main > div > div > div > div:nth-child(8) > div')

In [420]:
print(target_universities)

[['Michigan State University', 'United States', 'Masters Envir & Social System Modeling', 'See all admits', 'Get Guidance', ''], ['University of Iowa', 'United States', 'Masters Informatics', 'See all admits', 'Get Guidance', ''], ['University of Alabama at Birmingham', 'United States', 'MS Computer Science', '$31,980/year', '18 months', 'See all admits', 'Get Guidance', ''], ['Florida State University', 'United States', 'MS Information Technology', '$37,621/year', '18 months', 'See all admits', 'Get Guidance', ''], ['Rutgers University-New Brunswick', 'United States', 'MS Computer Science - Massive Data Analytics', '$37,680/year', '24 months', 'See all admits', 'Get Guidance', ''], ['Rutgers University, Newark', 'United States', 'MS Computer Science', '$38,534/year', '12 months', 'See all admits', 'Get Guidance', ''], ['Rutgers University-New Brunswick', 'United States', 'MS Computer Science', '$46,656/year', '24 months', 'See all admits', 'Get Guidance', ''], ['University of Cincinna

In [436]:
def scroll_to_top_of_page():
    top = driver.find_element(By.CSS_SELECTOR, '#__layout > div > div.w-full > div > main > div > div > div > div.pb-5.text-center.lg\:text-left > div')
    driver.execute_script("return arguments[0].scrollIntoView(true);", top)

In [438]:
scroll_to_top_of_page()

In [490]:
close_the_chat_box()

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x01087413+2389011]
	Ordinal0 [0x01019F61+1941345]
	Ordinal0 [0x00F0C658+837208]
	Ordinal0 [0x00F01687+792199]
	Ordinal0 [0x00F02752+796498]
	Ordinal0 [0x00EFC0D9+770265]
	Ordinal0 [0x00F0D9D0+842192]
	Ordinal0 [0x00F63AE2+1194722]
	Ordinal0 [0x00F53F66+1130342]
	Ordinal0 [0x00F2E546+976198]
	Ordinal0 [0x00F2F456+980054]
	GetHandleVerifier [0x01239632+1727522]
	GetHandleVerifier [0x012EBA4D+2457661]
	GetHandleVerifier [0x0111EB81+569713]
	GetHandleVerifier [0x0111DD76+566118]
	Ordinal0 [0x01020B2B+1968939]
	Ordinal0 [0x01025988+1989000]
	Ordinal0 [0x01025A75+1989237]
	Ordinal0 [0x0102ECB1+2026673]
	BaseThreadInitThunk [0x76D26A14+36]
	RtlInitializeExceptionChain [0x7758AB2F+143]
	RtlInitializeExceptionChain [0x7758AAFA+90]


In [ ]:
#__layout > div > main > div > div > div > validationobserver > form > div.grid.grid-cols-2.gap-6.pt-12 > div.text-right > button

In [446]:
top = driver.find_element(By.CSS_SELECTOR, '#__layout > div > main > div > div > div > validationobserver > form > div:nth-child(1) > div.-mx-4.sm\:-mx-12.border-b-2 > div > div.flex.justify-between.space-x-6.sm\:mr-12.mt-4.lg\:mt-0 > div.text-gray-700.hidden.sm\:block')
print(top.text)

Step 2 of 4
